Ch.4 word2vec속도 개선

3단원에서 구현한 Word2Vec는 말뭉치가 커질 경우 2가지 계산에서 병목을 일으킴.

1. 입력층의 원핫 표현과 가중치 행렬W_in의 곱 계산

2. 은닉층과 가중치행렬 W_out의 곱 및 softmax 계층의 계산 

첫 번째 문제점에 대한 개선 방법으로 Embedding 계층을 제안

해당 단어에 대한 원핫 벡터로 행렬곱을 하는 것이 아니라 index로 해당단어에 대한 분산 표현만 추출하는 방법

그러나 backward구현에 문제점이 있기 떄문에 먼저 쓰여진 값이 덮어쓰여지지 않도록 더하기로 바꾸어줌.(input이 맥락으로 들어가기 때문에 같은 단어가 발생할 수 있기 때문에)



두 번째 문제점에 대한 개선 방법으로 네거티브 샘플링 기법 사용 

다중 분류를 사용하면 계산량이 너무 많으므로 index로 특정 단어에 대한 분산 표현을 W_out으로부터 뽑아낸 다음 은닉층의 값과 dot_product를 진행하여 해당 단어에 대한 score를 sigmoid를 사용하여 값을 구해낸다.

의문점 : idx를 사용해버리면 inference단계는 어떻게 진행해야 하나?
지금 이 학습 방법은 단어의 분산표현을 얻기 위한 것이므로 상관없음. 또 inference는 많은 양의 말뭉치를 사용하지 않을 것이므로 상관없음. inference단계에서만 softmax사용(추측)

네거티브 샘플링에서는 부정적 예시를 학습할 필요가 있다. 그래서 
정답이 아닌 예시를 은닉층이후에 넣고 나온 loss값을 바른 예시로 나온 loss값과 더해주어 최종 loss값에 반영한다. 

부정적 예시를 샘플링하는 방법 

말뭉치의 통계데이터를 기반으로 샘플링(자주 등장하는 단어 일수록 선택될 가능성이 높음.) => numpy.random.choice()사용

출현확률이 낮은 단어를 버리지 않기 위해서 0.75를 제곱함. => 식 4.4 참고 


단어의 분산 표현의 장점 : 단어를 고정 길이 벡터로 변환시켜 준다는 것

모델에 따라 정확도가 다르다.

일반적으로 말뭉치가 클수록 결과가 좋다.

단어 벡터의 차원수는 적당한 크기가 좋다.